# 🤝 Enterprise Multi-Agent Workflow Systems (.NET)

## 📋 Learning Objectives

This notebook demonstrates how to build sophisticated enterprise-grade multi-agent systems using the Microsoft Agent Framework in .NET with GitHub Models. You'll learn to orchestrate multiple specialized agents working together through structured workflows, leveraging .NET's enterprise features for production-ready solutions.

**Enterprise Multi-Agent Capabilities You'll Build:**
- 👥 **Agent Collaboration**: Type-safe agent coordination with compile-time validation
- 🔄 **Workflow Orchestration**: Declarative workflow definition with .NET's async patterns
- 🎭 **Role Specialization**: Strongly-typed agent personalities and expertise domains
- 🏢 **Enterprise Integration**: Production-ready patterns with monitoring and error handling


## ⚙️ Prerequisites & Setup

**Development Environment:**
- .NET 9.0 SDK or higher
- Visual Studio 2022 or VS Code with C# extension
- Azure subscription (for persistent agents)

**Required NuGet Packages:**
```xml
<PackageReference Include="Microsoft.Extensions.AI.Abstractions" Version="9.9.0" />
<PackageReference Include="Azure.AI.Agents.Persistent" Version="1.2.0-beta.4" />
<PackageReference Include="Azure.Identity" Version="1.15.0" />
<PackageReference Include="System.Linq.Async" Version="6.0.3" />
<PackageReference Include="Microsoft.Extensions.AI" Version="9.8.0" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
```



In [1]:
#r "nuget: Microsoft.Extensions.Logging.Abstractions, 10.0.0"
#r "nuget: Microsoft.Extensions.DependencyInjection.Abstractions, 10.0.0"
#r "nuget: Microsoft.Extensions.Options, 10.0.0"
#r "nuget: OpenAI, 2.8.0"

Installed Packages Microsoft.Extensions.DependencyInjection.Abstractions, 10.0.0 Microsoft.Extensions.Logging.Abstractions, 10.0.0 Microsoft.Extensions.Options, 10.0.0 OpenAI, 2.8.0

In [2]:
// 📦 NuGet Package Reference: Microsoft Extensions AI Abstractions
// Core AI abstraction layer providing standardized interfaces for multi-agent systems
// Essential for building type-safe, enterprise-grade agent orchestration
#r "nuget: Microsoft.Extensions.AI.Abstractions, 10.2.0"

Installed Packages Microsoft.Extensions.AI.Abstractions, 10.2.0

In [3]:
// 📦 Enterprise Multi-Agent Dependencies
// Azure.AI.Agents.Persistent: Enterprise agent management with state persistence and Azure integration
// Azure.Identity: Secure authentication for Azure services in multi-agent workflows
// System.Linq.Async: High-performance asynchronous LINQ operations for agent data processing
// Microsoft.Extensions.AI: Core AI service abstractions for agent coordination
// DotNetEnv: Secure environment variable management for multi-agent configuration
#r "nuget: System.ClientModel, 1.8.1"
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.8"
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: System.Linq.Async, 7.0.0"
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: OpenTelemetry.Api, 1.14.0"

Installed Packages Azure.AI.Agents.Persistent, 1.2.0-beta.8 Azure.Identity, 1.18.0-beta.2 DotNetEnv, 3.1.1 OpenTelemetry.Api, 1.14.0 system.clientmodel, 1.8.1 System.Linq.Async, 7.0.0

In [4]:
#r "nuget: Microsoft.Extensions.AI, 10.2.0"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 10.2.0-preview.1.26063.2"

Installed Packages Microsoft.Extensions.AI, 10.2.0 Microsoft.Extensions.AI.OpenAI, 10.2.0-preview.1.26063.2

In [5]:

// #r "nuget: Microsoft.Agents.AI.Workflows, 1.0.0-preview.260108.1"
#r "/Users/lokinfey/Desktop/AOAI/Foundry/agent-framework/dotnet/src/Microsoft.Agents.AI.Workflows/bin/Debug/net10.0/Microsoft.Agents.AI.Workflows.dll"

In [6]:
// 🔗 Local Assembly Reference: Microsoft Agents AI OpenAI
// OpenAI-compatible integration for GitHub Models in multi-agent systems
// Enables high-performance language model access for agent communication and reasoning
// #r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.260108.1"
#r "/Users/lokinfey/Desktop/AOAI/Foundry/agent-framework/dotnet/src/Microsoft.Agents.AI.OpenAI/bin/Debug/net10.0/Microsoft.Agents.AI.OpenAI.dll"

In [7]:
// 🔄 Local Assembly Reference: Microsoft Agents Workflows
// Advanced workflow orchestration engine for complex multi-agent coordination
// Provides declarative workflow definition and type-safe agent interaction patterns
// #r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.2"

In [8]:
// 🔌 NuGet Package Reference: OpenAI Integration (Preview)
// Preview version providing GitHub Models compatibility for multi-agent workflows
// Enables seamless integration with GitHub's model inference service for agent communication
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [9]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [10]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;
using OpenAI;

In [11]:
 using DotNetEnv;

In [12]:
Env.Load("../../../.env");

In [13]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

## 🔑 Load GitHub Models Configuration

Extract and validate the required API credentials from environment variables.

In [14]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

## ⚙️ Configure OpenAI Client Options

Set up the OpenAI client to use GitHub Models endpoint.

In [15]:
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

## 🔌 Initialize OpenAI Client

Create the authenticated client for GitHub Models communication.

In [16]:
const string REVIEWER_NAME = "Concierge";
const string REVIEWER_INSTRUCTIONS = @"""
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """;

## 🎩 Define Concierge Agent (Quality Reviewer)

Configure the Concierge agent who evaluates travel recommendations for local authenticity and provides feedback for refinement.

In [17]:
const string FRONTDESK_NAME = "FrontDesk";
const string FRONTDESK_INSTRUCTIONS = @"""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """;

## 🏨 Define Front Desk Agent (Travel Specialist)

Configure the Front Desk agent who provides expert, concise travel recommendations.

In [18]:

ChatClientAgentOptions frontdeskAgentOptions = new(){
    Name = FRONTDESK_NAME,
    Description = FRONTDESK_INSTRUCTIONS
};
ChatClientAgentOptions reviewerAgentOptions = new(){
    Name = REVIEWER_NAME,
    Description = REVIEWER_INSTRUCTIONS
};

## ⚙️ Configure Agent Options

Create strongly-typed agent configuration options for both specialized agents.

In [19]:
using Microsoft.Extensions.AI;

AIAgent reviewerAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).AsIChatClient().AsAIAgent(
    reviewerAgentOptions);
AIAgent frontdeskAgent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).AsIChatClient().AsAIAgent(
    frontdeskAgentOptions);

## 🤖 Create Agent Instances

Instantiate both specialized agents with their configured options.

In [20]:
var workflow = new WorkflowBuilder(frontdeskAgent)
            .AddEdge(frontdeskAgent, reviewerAgent)
            .Build();

## 🔄 Build the Multi-Agent Workflow

Create the workflow pipeline using `WorkflowBuilder`:
1. **Start**: Front Desk agent provides initial recommendation
2. **Edge**: Concierge agent reviews and refines for authenticity

In [21]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, new ChatMessage(ChatRole.User, "I would like to go to Paris."));

## 🚀 Start the Streaming Workflow Execution

Initialize the streaming run for the multi-agent workflow with the user's travel request.

In [22]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));

## 📨 Send Message to Start Workflow

Trigger the workflow execution by sending the initial message with event emission enabled.

In [23]:
string strResult = "";

## 📝 Initialize Result Accumulator

Create a string to collect the full workflow output.

In [24]:
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
            if (evt is AgentResponseUpdateEvent executorComplete)
            {
                strResult += executorComplete.Data;
                Console.WriteLine($"{executorComplete.ExecutorId}: {executorComplete.Data}");
            }
    // Console.WriteLine(evt);
}

FrontDesk_1daa382177734244a831be7a92563c57: 
FrontDesk_1daa382177734244a831be7a92563c57: 
FrontDesk_1daa382177734244a831be7a92563c57: That
FrontDesk_1daa382177734244a831be7a92563c57:  sounds
FrontDesk_1daa382177734244a831be7a92563c57:  wonderful
FrontDesk_1daa382177734244a831be7a92563c57: !
FrontDesk_1daa382177734244a831be7a92563c57:  Paris
FrontDesk_1daa382177734244a831be7a92563c57:  is
FrontDesk_1daa382177734244a831be7a92563c57:  a
FrontDesk_1daa382177734244a831be7a92563c57:  beautiful
FrontDesk_1daa382177734244a831be7a92563c57:  city
FrontDesk_1daa382177734244a831be7a92563c57:  with
FrontDesk_1daa382177734244a831be7a92563c57:  so
FrontDesk_1daa382177734244a831be7a92563c57:  much
FrontDesk_1daa382177734244a831be7a92563c57:  to
FrontDesk_1daa382177734244a831be7a92563c57:  offer
FrontDesk_1daa382177734244a831be7a92563c57: .
FrontDesk_1daa382177734244a831be7a92563c57:  Are
FrontDesk_1daa382177734244a831be7a92563c57:  you
FrontDesk_1daa382177734244a831be7a92563c57:  thinking
FrontDesk_1d

## 🔄 Process Streaming Workflow Events

Watch the stream and process events as each agent completes their contribution.
The output shows which agent (executor) is responding and their message content.

In [25]:
strResult

That sounds wonderful! Paris is a beautiful city with so much to offer. Are you thinking about visiting any specific places there, like the Eiffel Tower, the Louvre, or maybe trying some French cuisine? I can help you plan your trip or provide some tips if you'd like!That sounds wonderful! Paris is a fantastic city with so much to see and do. Are you thinking of visiting any specific spots like the Eiffel Tower, Louvre Museum, or Montmartre? I can help you with sightseeing suggestions, restaurant recommendations, or travel tips if you'd like!

## 📖 Display Final Aggregated Result

Show the complete output from the multi-agent collaboration.